In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

In [2]:
df = pd.read_csv('/Users/rushabhpatel/Desktop/Master/Enero/cnn/train.csv')

In [3]:
df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [4]:
df.shape

(287113, 3)

In [5]:
df.columns

Index(['id', 'article', 'highlights'], dtype='object')

In [6]:
df.info()    #No tenemos nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          287113 non-null  object
 1   article     287113 non-null  object
 2   highlights  287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB


In [7]:
df.count()

id            287113
article       287113
highlights    287113
dtype: int64

In [8]:
df2 = df.sample(frac=0.05, random_state=42).copy()


In [9]:
df2.count()

id            14356
article       14356
highlights    14356
dtype: int64

In [10]:
df2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 14356 entries, 272581 to 240397
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          14356 non-null  object
 1   article     14356 non-null  object
 2   highlights  14356 non-null  object
dtypes: object(3)
memory usage: 448.6+ KB


In [11]:
df2['article_length'] = df2['article'].apply(len)

In [12]:
pd.set_option('display.max_colwidth', None)  # Muestra todo el contenido de una celda, sin truncar el texto

df.iloc[[0],[2]]

,highlights
0,"Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed ."


In [13]:
pd.reset_option('display.max_colwidth')


Preprocesamiento


In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
def remove_stopwords(text):
    word_tokens = word_tokenize(text.lower())  # Convertir a minúsculas
    filtered_words = [word for word in word_tokens if word not in stop_words and word.isalnum()]
    return ' '.join(filtered_words)

df2['article_no_stopwords'] = df2['article'].apply(remove_stopwords)

print(df2[['article', 'article_no_stopwords']].head())

                                                  article  \
272581  By . Mia De Graaf . Britons flocked to beaches...   
772     A couple who weighed a combined 32st were sham...   
171868  Video footage shows the heart stopping moment ...   
63167   Istanbul, Turkey (CNN) -- About 250 people rac...   
68522   By . Daily Mail Reporter . PUBLISHED: . 12:53 ...   

                                     article_no_stopwords  
272581  mia de graaf britons flocked beaches across so...  
772     couple weighed combined 32st shamed slimming f...  
171868  video footage shows heart stopping moment 17 y...  
63167   istanbul turkey cnn 250 people raced across sy...  
68522   daily mail reporter published est 3 january 20...  


In [16]:
df2['highlights_no_stopwords'] = df2['highlights'].apply(remove_stopwords)

print(df2[['highlights', 'highlights_no_stopwords']].head())

                                               highlights  \
272581  People enjoyed temperatures of 17C at Brighton...   
772     Couple started piling on pounds after the birt...   
171868  A 17-year-old boy suffering lacerations to his...   
63167   Syrians citizens hightail it to Turkey .\nMost...   
68522   The Xue Long had provided the helicopter that ...   

                                  highlights_no_stopwords  
272581  people enjoyed temperatures 17c brighton beach...  
772     couple started piling pounds birth two childre...  
171868  boy suffering lacerations left hand fingers sa...  
63167   syrians citizens hightail turkey ethnic turkme...  
68522   xue long provided helicopter helped rescue cre...  


In [17]:
df2.head()

,id,article,highlights,article_length,article_no_stopwords,highlights_no_stopwords
272581,ed0fed726929c1eeabe6c390e47128dbb7d7a055,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...,3735,mia de graaf britons flocked beaches across so...,people enjoyed temperatures 17c brighton beach...
772,023cd84001b33aed4ff0f3f5ecb0fdd2151cf543,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...,3638,couple weighed combined 32st shamed slimming f...,couple started piling pounds birth two childre...
171868,6a70a0d8d3ed365fe1df6d35f1587a8b9b298618,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...,4880,video footage shows heart stopping moment 17 y...,boy suffering lacerations left hand fingers sa...
63167,b37204c13ea38b511265e41ac69fb12acfb63f85,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...,1484,istanbul turkey cnn 250 people raced across sy...,syrians citizens hightail turkey ethnic turkme...
68522,c24e5805afd5145bc48410e876db91d44a06be5e,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...,6430,daily mail reporter published est 3 january 20...,xue long provided helicopter helped rescue cre...


In [18]:
df2.article_no_stopwords

272581    mia de graaf britons flocked beaches across so...
772       couple weighed combined 32st shamed slimming f...
171868    video footage shows heart stopping moment 17 y...
63167     istanbul turkey cnn 250 people raced across sy...
68522     daily mail reporter published est 3 january 20...
                                ...                        
276742    al qaeda planned hijack cruise ships post foot...
108480    cnn circus producer said friday animal rights ...
53210     new delhi india cnn india successfully launche...
204009    clear action taken oxford union president ben ...
240397    unlike men going dean kremer go get drunk mate...
Name: article_no_stopwords, Length: 14356, dtype: object

In [19]:
lista = []

for a in df2.article_no_stopwords:
    lista.append(a)
    

In [20]:
lista2 = []
for a in df2.highlights_no_stopwords:
    lista2.append(a)

In [21]:
len(lista)

14356

In [22]:
len(lista2)

14356

In [25]:
texto_completo = " ".join(lista)


tabla_traduccion = str.maketrans('', '', string.punctuation)
palabras_limpias = [palabra.translate(tabla_traduccion).lower() for palabra in texto_completo.split()]

frecuencia = Counter(palabras_limpias)


In [36]:
len(palabras_limpias)

425596

In [26]:
print(frecuencia.most_common(50)) 


[('said', 65616), ('one', 23208), ('would', 19225), ('also', 17845), ('people', 17618), ('new', 16892), ('two', 15677), ('last', 14786), ('told', 14482), ('time', 14180), ('police', 13783), ('could', 13728), ('first', 13474), ('years', 13370), ('year', 12552), ('mr', 12164), ('like', 11183), ('home', 10772), ('left', 10048), ('world', 10025), ('back', 9976), ('family', 9518), ('get', 9184), ('three', 8901), ('made', 8862), ('cnn', 8823), ('found', 8502), ('day', 8440), ('right', 8175), ('says', 7889), ('may', 7718), ('court', 7637), ('around', 7617), ('even', 7579), ('many', 7435), ('man', 7150), ('make', 7110), ('since', 7082), ('according', 7076), ('life', 7048), ('children', 7013), ('way', 6924), ('take', 6743), ('going', 6672), ('still', 6531), ('house', 6478), ('city', 6476), ('say', 6418), ('see', 6356), ('week', 6347)]


In [27]:
texto_completo2 = " ".join(lista2)


tabla_traduccion = str.maketrans('', '', string.punctuation)
palabras_limpias = [palabra.translate(tabla_traduccion).lower() for palabra in texto_completo2.split()]

frecuencia2 = Counter(palabras_limpias)




In [28]:
frecuencia2.most_common(50)

[('says', 3525),
 ('new', 2224),
 ('said', 1966),
 ('police', 1487),
 ('two', 1435),
 ('one', 1425),
 ('also', 1243),
 ('years', 1186),
 ('last', 1138),
 ('could', 1103),
 ('first', 1103),
 ('found', 1090),
 ('say', 1055),
 ('people', 1031),
 ('year', 1017),
 ('home', 961),
 ('world', 938),
 ('three', 905),
 ('would', 796),
 ('family', 717),
 ('man', 694),
 ('since', 683),
 ('made', 681),
 ('back', 659),
 ('time', 657),
 ('former', 647),
 ('children', 641),
 ('court', 631),
 ('city', 603),
 ('died', 597),
 ('four', 595),
 ('week', 581),
 ('president', 568),
 ('claims', 554),
 ('death', 548),
 ('told', 543),
 ('day', 541),
 ('show', 539),
 ('may', 524),
 ('mother', 524),
 ('obama', 524),
 ('left', 523),
 ('house', 522),
 ('used', 521),
 ('killed', 518),
 ('women', 503),
 ('five', 478),
 ('set', 471),
 ('united', 470),
 ('hospital', 467)]

In [33]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(lista2)
num_clusters = 5  # Ajusta este valor según tus necesidades
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

clusters = kmeans.predict(X)


## Visualización
#tsne = TSNE(n_components=2, random_state=42)
#X_tsne = tsne.fit_transform(X.toarray())
#
## Crear gráfico de dispersión
#plt.figure(figsize=(8, 6))
#plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=clusters, cmap='viridis')
#plt.title("Clusterización de Artículos")
#plt.xlabel("t-SNE 1")
#plt.ylabel("t-SNE 2")
#plt.colorbar(label='Cluster')
#plt.show()


/Users/rushabhpatel/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [40]:
df2["cluster"] = clusters

In [41]:
clusters

array([2, 2, 2, ..., 2, 0, 2], dtype=int32)

In [43]:
df2.head()

,id,article,highlights,article_length,article_no_stopwords,highlights_no_stopwords,cluster
272581,ed0fed726929c1eeabe6c390e47128dbb7d7a055,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...,3735,mia de graaf britons flocked beaches across so...,people enjoyed temperatures 17c brighton beach...,2
772,023cd84001b33aed4ff0f3f5ecb0fdd2151cf543,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...,3638,couple weighed combined 32st shamed slimming f...,couple started piling pounds birth two childre...,2
171868,6a70a0d8d3ed365fe1df6d35f1587a8b9b298618,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...,4880,video footage shows heart stopping moment 17 y...,boy suffering lacerations left hand fingers sa...,2
63167,b37204c13ea38b511265e41ac69fb12acfb63f85,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...,1484,istanbul turkey cnn 250 people raced across sy...,syrians citizens hightail turkey ethnic turkme...,4
68522,c24e5805afd5145bc48410e876db91d44a06be5e,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...,6430,daily mail reporter published est 3 january 20...,xue long provided helicopter helped rescue cre...,2


In [54]:
articulos_cluster_0 = [lista2[i] for i in range(len(lista2)) if clusters[i] == 4]

print("Artículos del Cluster 2:")
for i, articulo in enumerate(articulos_cluster_0[:10]):  # Limitar a 3 artículos
    print(f"\nArtículo {i+1} del Cluster 2:")
    print(articulo)
    print("\n" + "="*80 + "\n")


Artículos del Cluster 2:

Artículo 1 del Cluster 2:
syrians citizens hightail turkey ethnic turkmens official says turkey wonders tip iceberg



Artículo 2 del Cluster 2:
lil wayne lyrics include pepsico calls offensive reference emmett till till black youth chicago murdered mississippi 1955 publicist rapper label says artist company parted ways amicably



Artículo 3 del Cluster 2:
ellis cashmore professor staffordshire university says little proof olympic games create sporting legacy academic said sport school children fun competitive



Artículo 4 del Cluster 2:
adults children cambridge lecturer says lack play stops normal development



Artículo 5 del Cluster 2:
wide sinkhole creates millions damages national corvette museum cave says museum spokeswoman engineers say sky dome suffered structural damage cost damaged cars substantial museum director says



Artículo 6 del Cluster 2:
7 killed 20 hurt saturday muslim protesters set fire houses christian area protesting alleged desecra

In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(df2['article_no_stopwords'])
X = tokenizer.texts_to_sequences(df2['article_no_stopwords'])

X_pad = pad_sequences(X, maxlen=500)  

print(X_pad.shape)  


(14356, 500)


In [46]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(df2['cluster'])  


In [47]:


model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=X_pad.shape[1]))
model.add(SpatialDropout1D(0.2))  
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))  
model.add(Dense(5, activation='softmax')) 


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_pad, y, epochs=2, batch_size=64, validation_split=0.2)


Epoch 1/2
180/180 [==============================] - 249s 1s/step - loss: 1.0573 - accuracy: 0.6228 - val_loss: 0.8894 - val_accuracy: 0.7093
Epoch 2/2
180/180 [==============================] - 233s 1s/step - loss: 0.7975 - accuracy: 0.7066 - val_loss: 0.9080 - val_accuracy: 0.6807


In [48]:
predictions = model.predict(X_pad)

449/449 [==============================] - 71s 156ms/step


In [49]:
print(predictions[:10])  # Muestra las primeras 10 predicciones


[[3.5164814e-02 1.3759515e-02 8.7905252e-01 6.0504554e-03 6.5972760e-02]
 [4.4150401e-02 1.2082445e-02 8.5699844e-01 4.0914360e-03 8.2677349e-02]
 [2.0323366e-01 1.7541749e-02 6.4715028e-01 8.2164854e-03 1.2385779e-01]
 [3.3275034e-02 1.0761462e-02 8.4555060e-01 2.5212297e-03 1.0789163e-01]
 [1.3024342e-02 4.2038998e-03 9.4950134e-01 1.1582742e-03 3.2112204e-02]
 [4.8852339e-03 1.4576508e-02 1.0189760e-01 8.6945975e-01 9.1809379e-03]
 [2.4562314e-02 6.8666590e-03 9.1052127e-01 1.8926674e-03 5.6156993e-02]
 [1.0393081e-02 2.6616666e-03 9.6192878e-01 6.2389864e-04 2.4392383e-02]
 [4.0537216e-02 1.8446546e-02 8.3053648e-01 6.6429796e-03 1.0383679e-01]
 [1.2565650e-01 2.5775459e-02 6.0685039e-01 7.5793457e-03 2.3413821e-01]]


In [50]:
import numpy as np

np.argmax(predictions[:10],axis = 1)


array([2, 2, 2, 2, 2, 3, 2, 2, 2, 2])